# 1. Hashing task!

In [1]:
import numpy as np
import random
import time

### First of all we need to find (n) the number of all the elements which are going to be inserted in the bloom filter.

In [2]:
pool = open("passwords1.txt")
n = 0
while(pool.readline() != ""):
    n += 1
pool.close()

In [3]:
print(n)

100000000


### From the bloom filter's wikipedia page we can see that the optimal number of hash functions can be attained from these formulas:
### $k = \frac{m}{n} ln(2)$
### $m = - \frac{n (lnp)}{(ln2)^2}$
##### k = The number of hash functions.<br>n = The number of inserted elements.<br>m = Size of the bloom filter.<br>p = False positive rate.

In [4]:
# here we take p = 0.01 and find corresponding m:
p = 0.05
m = -1*n*np.log(p)/(np.log(2))**2
print(m)

623522422.9572682


#### We want the size of our bloom filter to be a prime number so we can use it in our hash function, so we will find the closest prime number to this:

In [5]:
def is_prime(a):
    ub = np.floor(int(np.sqrt(a)))
    for i in range(2 ,int(ub)):
        if a % i == 0:
            return False
    return True

def closest_prime(a):
    i = 0
    a = np.floor(a)
    while(True):
        if(is_prime(a +i)):
            return int(a + i)
        if( m -i > 0 and is_prime(a -i)):
            return int(a - i)
        i += 1

m = closest_prime(m)
print(m)

623522423


#### Now that we have the size of our bloom filter, we can find the optimal number of hash functions:

In [6]:
k = m * np.log(2) / n
k = int(round(k))
print(k)

4


### It's time to create our hash functions
#### We need 20 random coefficients for each hash function:

In [7]:
coeffs = []
for i in range(k):
    a = []
    for j in range(20):
        a.append(random.randrange(m))
    coeffs.append(a)
print(coeffs)
        

[[137764682, 314396409, 353945432, 75742771, 309245389, 18300369, 272098438, 40734884, 56208480, 33011002, 317609153, 582008730, 329017868, 298235097, 592170620, 283341426, 337819242, 55136982, 298203738, 234778959], [492398237, 529394535, 270980423, 385778073, 172099484, 512972309, 87370041, 63574328, 233832667, 206353444, 432606583, 129495094, 440497579, 181275207, 477360785, 522286155, 611343192, 122219416, 57299481, 237780027], [571923885, 302176175, 197234184, 342650466, 523602128, 359926323, 229552571, 37271860, 61547816, 337345667, 268219831, 233733500, 365994133, 557121759, 499169399, 479601089, 140454426, 87488479, 295977027, 96147671], [553424247, 37761661, 485554170, 603090137, 69506933, 306857497, 350832038, 540896445, 444856890, 442407900, 425931361, 162983597, 236613639, 420498737, 87324404, 14297922, 402698035, 119776982, 618053139, 92108484]]


In [8]:
def my_hash(m, coeffs, string):
    h = 0
    for i in range(20):
        h += ord(string[i]) * coeffs[i]
    return h%m

### Now that our hash functions are ready, we can fill out the bloom filter:

In [9]:
bloom_filter = []
for i in range(m):
    bloom_filter.append(False)

In [10]:
pool = open("passwords1.txt")
start = time.time()
for i in range(n):
    s = pool.readline()
    for j in range(k):
        key = my_hash(m, coeffs[j], s)
        bloom_filter[key] = True
pool.close()

### Now that the bloom filter is available we can get the results for our test set.

In [11]:
duplicate = 0 #counter for duplicate passwords
test = open("passwords2.txt")
s = test.readline()
while(s != ""):
    flag = True
    for i in range(k):
        key = my_hash(m, coeffs[i], s)
        if( bloom_filter[key] == False ):
            flag = False
            break
    if(flag):
        duplicate += 1
    s = test.readline()
end = time.time()
test.close()
print(duplicate, " password(s) are present in the filter.")

15257184  password(s) are present in the filter.


In [13]:
print('Number of hash function used: ', k)
print('Number of duplicates detected: ', duplicate)
print('Probability of false positives: around ', p)
print('Execution time: ', end-start)

Number of hash function used:  4
Number of duplicates detected:  15257184
Probability of false positives: around  0.05
Execution time:  2782.8581113815308
